Create a jupyter notebook template to do the following - for a given token and date range:
1. calculate the hourly return by timestamp
2. calculate the hourly volatility (you can use the formula we went over in class)
3. calculate maximum drawdown up to that hour for each hour
4. then plot in a 2x2 grid:hourly return, hourly vol, maximum drawdown, close prices, color each day on the graph

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import scrapbook as sb 
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.dates import date2num, DateFormatter

In [ ]:
plt.style.use('ggplot')
plt.tight_layout()
plt.rcParams['figure.figsize'] = [20, 20]

In [ ]:
%matplotlib inline

In [ ]:
token = 'BTC'
from_date = '2021-11-05'
to_date = '2021-11-12'
connection_string = 'sqlite:///data/data.db'

In [ ]:
prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc 
WHERE 
    token = '{token}'
    AND ts >= '{from_date}'
    AND ts < '{to_date}'
''', connection_string)
prices['ts'] = pd.to_datetime(prices['ts'])

prices.set_index('ts', inplace=True)

In [ ]:
prices

Here I used the same volatility definition as the one in the market analysis template. However, I set the lookback period to be 3.

In [ ]:
def vol_ohlc(df, lookback=3):
    o = df.open
    h = df.high
    l = df.low
    c = df.close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result[:lookback-1] = np.nan
    
    return result

In [ ]:
def series_plot(data, ax=None):
    data.plot(ax=ax, legend=False)
    ax.set_title(label=list(data.columns)[0], loc='left')
    start_of_days = data.between_time('00:00:00', '00:00:00').index.to_list()
    end_of_days = start_of_days[1:] + [data.index[-1]]
    colors = iter(cm.rainbow(np.linspace(0, 1, len(start_of_days))))
    
    for x, y in zip(start_of_days, end_of_days):
        color = next(colors)
        ax.axvspan(x, y, color=color, alpha=0.1)

In [ ]:
prices['volatility']=vol_ohlc(prices)

In [ ]:
prices['hourly return'] = np.log(prices.close)-np.log(prices.open)

In [ ]:
prices['hourly drawdown'] = prices.high-prices.close

In [ ]:
prices['maximum drawdown']=0
for i in range(prices.shape[0]):
     prices['maximum drawdown'].iloc[i]=prices['hourly drawdown'].iloc[0:(i+1)].max()

In [ ]:
%%capture

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 20))

In [ ]:
series_plot(prices[['hourly return']], ax=ax[0,0])
series_plot(prices[['volatility']], ax=ax[1,0])
series_plot(prices[['maximum drawdown']], ax=ax[0,1])
series_plot(prices[['close']], ax=ax[1,1])
fig.suptitle(f'Market action for {token} between {from_date} and {to_date}')
fig

In [ ]:
sb.glue('token', token)
sb.glue('from_date', from_date)
sb.glue('to_date', to_date)
sb.glue('prices', prices, encoder='pandas')
sb.glue('summary_plot', fig, encoder='display', display=True)